In [1]:
import pandas as pd

filepath = 'Imports/OutcomeData/2010CensusData.csv'
census_data = pd.read_csv(filepath)

filepath = 'Exports/Data/5.CountyExposure.csv'
county_exposure = pd.read_csv(filepath)

In [2]:
print(f'Only {1- (census_data['COUNTYFIP'] == 0).mean() :.2%} of the rows are identified by county')

Only 56.08% of the rows are identified by county


In [3]:
merged_data = pd.merge(census_data, county_exposure, left_on=['STATEFIP','COUNTYFIP'], right_on=['STATEFP', 'COUNTYFP'], how='inner')

In [4]:
merged_data.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,COUNTYFIP,STRATA,GQ,...,COUNTYFP,weighted_ave_HH_conc,weighted_ave_non_HH_conc,weighted_sum_HH_conc,weighted_sum_non_HH_conc,non_weighted_ave_HH_conc,non_weighted_ave_non_HH_conc,non_weighted_sum_HH_conc,non_weighted_sum_non_HH_conc,initial_HH_exp
0,2010,201001,2,80,97.0,2010000000021,1,97,220001,1,...,97,0.205318,0.794682,2441.1,9448.275,0.205759,0.794241,343.0,1324.0,1974.0
1,2010,201001,2,80,97.0,2010000000021,1,97,220001,1,...,97,0.205318,0.794682,2441.1,9448.275,0.205759,0.794241,343.0,1324.0,1974.0
2,2010,201001,2,80,97.0,2010000000021,1,97,220001,1,...,97,0.205318,0.794682,2441.1,9448.275,0.205759,0.794241,343.0,1324.0,1974.0
3,2010,201001,4,224,82.0,2010000000041,1,117,130001,1,...,117,NaN,NaN,NaN,NaN,0.075431,0.924569,35.0,429.0,1974.0
4,2010,201001,4,224,82.0,2010000000041,1,117,130001,1,...,117,NaN,NaN,NaN,NaN,0.075431,0.924569,35.0,429.0,1974.0


In [5]:
# Removing detailed variables
detailed_cols = [col for col in merged_data.columns if col[-1:] == 'D']
print(detailed_cols)
merged_data.drop(columns=detailed_cols, inplace=True)

['RACED', 'HISPAND', 'BPLD', 'EDUCD', 'GRADEATTD', 'EMPSTATD']


In [6]:
# Removing values that are encoded at missing
import numpy as np

vals_to_remove = [9999998, 9999999, 999, 99, 9]
census_cols = [col for col in merged_data.columns if 'HH_conc' not in col]
print(merged_data[census_cols].map(lambda x: x in vals_to_remove).any(axis='rows'))
merged_data = merged_data.map(lambda x: np.nan if x in vals_to_remove else x)
assert not merged_data[census_cols].map(lambda x: x in vals_to_remove).any().any()

YEAR              False
SAMPLE            False
SERIAL             True
CBSERIAL           True
HHWT               True
CLUSTER           False
STATEFIP           True
COUNTYFIP          True
STRATA            False
GQ                False
RENTGRS            True
HHINCOME           True
VALUEH             True
NMOTHERS          False
NFATHERS          False
PERNUM             True
PERWT              True
SEX               False
AGE                True
MARST             False
RACE               True
HISPAN            False
BPL                True
EDUC              False
GRADEATT          False
SCHLTYPE          False
EMPSTAT           False
INCTOT             True
STATEFP            True
COUNTYFP           True
initial_HH_exp    False
dtype: bool


In [7]:
filepath = 'Exports/Data/6.CountyMasterDataset.csv'
merged_data.to_csv(filepath, index=False)